In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import seaborn as sns

In [ ]:
data=pd.read_csv('/kaggle/input/engineering-placements-prediction/collegePlace.csv')

In [ ]:
#checking for null values
data.isnull().sum()

In [ ]:
#finding categorial column
categorial_column=list(data.select_dtypes(include=['O']).columns)

In [ ]:
sns.barplot(x='Internships',y='PlacedOrNot',data=data)

In [ ]:
sns.barplot(x='CGPA',y='PlacedOrNot',data=data)

In [ ]:
sns.barplot(x='PlacedOrNot',y='CGPA',data=data)

In [ ]:
sns.barplot(x='HistoryOfBacklogs',y='PlacedOrNot',data=data)

In [ ]:
label=LabelEncoder()
data['Gender']=label.fit_transform(data['Gender'])
dumies=pd.get_dummies(data['Stream'])
data=pd.concat([data,dumies],axis='columns')

In [ ]:
data.drop('Stream',axis=1,inplace=True)

In [ ]:
plt.figure(figsize=(20,10))
data_corr=data.corr()
sns.heatmap(data_corr,annot=True)

In [ ]:
data.describe()

In [ ]:
# spliting the data into cv set and testing set
percent_for_cv=len(data)-int(2966*0.1)
df_cv=data.head(2670)
df_test=data[data.index>2670]

In [ ]:
model_param={
    'LogisticRegression':{
        'model':LogisticRegression(solver='liblinear'),
        'params':{
            'C':[1.0,2.0,3.0,10.0,15.0,20.0]
        }
    },
    'SVC':{
        'model':SVC(gamma='auto',),
        'params':{
            'kernel':['linear','rbf','poly'],
            'C':[1.0,2.0,3.0,10.0,15.0,20.0]
        }
    }
}
x=df_cv.drop('PlacedOrNot',axis=1)
y=df_cv['PlacedOrNot']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.1,shuffle=True,random_state=1)

score=[]
for model_name,param in model_param.items():
  rscv=RandomizedSearchCV(param['model'],param['params'],n_iter=5,cv=5,return_train_score=False)
  rscv.fit(xtrain,ytrain)
  score.append({
    'model':model_name,
    'best_score':rscv.best_score_,
    'best_param':rscv.best_params_
  })

In [ ]:
#with the help of score we will come to knpow the best model and parameters that best fits the data
score

In [ ]:
#svc give higher accuracy than logisticregression
df_test=df_test.sample(frac=1)
x_test=df_test.drop('PlacedOrNot',axis=1)
y_test=df_test['PlacedOrNot']
sv=SVC(C= 10.0, kernel= 'rbf',gamma='auto')
sv.fit(xtrain,ytrain)
pred=sv.predict(x_test)
acc=accuracy_score(y_test,pred)
acc